# Instalação e Importação

In [1]:
!pip install geopandas pandas requests sqlalchemy psycopg2-binary shapely matplotlib overpy osmnx folium owslib arcgis

Defaulting to user installation because normal site-packages is not writeable
  Using cached geopandas-1.1.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached pandas-2.3.1-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached sqlalchemy-2.0.41-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached psycopg2_binary-2.9.10-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.9 kB)
  Using cached shapely-2.1.1-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached matplotlib-3.10.3-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached overpy-0.7-py3-none-any.whl.metadata (3.5 kB)
  Using cached osmnx-2.0.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached folium-0.20.0-py2.py3-none-any.whl.metadata (4.2 kB)
  Using cached owslib-0.34.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached arcgis-2.0.0.tar.gz (3.7 MB)
  Installing 

In [3]:
%pip install pandas requests matplotlib geopandas numpy seaborn

import pandas as pd
import requests
import matplotlib.pyplot as plt
import os
import zipfile
import geopandas as gpd
import numpy as np
import seaborn as sns

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-2.3.1-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached matplotlib-3.10.3-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached geopandas-1.1.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached numpy-2.3.1-cp313-cp313-manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached contourpy-1.3.2-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.58.5-cp313-cp313-manylinux1_x86_64.manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_5_x86_64.whl.metadata (106 kB)
  Using cached kiwisolver-1.4.8-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.2 kB)

# Análise de dados chuva e óbitos na RMR

### ⚙️ Pipeline de Integração: Mortalidade e Pluviometria na RMR (PE)

In [8]:
# ========================================================
#                  CONFIGURAÇÕES INICIAIS
# ========================================================

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (16, 8)
sns.set_palette("husl")

# ========================================================
#               CARREGAMENTO E PREPARAÇÃO DOS DADOS
# ========================================================

def carregar_dados_mortalidade():
    """Carrega e processa os dados de mortalidade"""
    print("\n📂 CARREGANDO DADOS DE MORTALIDADE")
    print("="*50)

    dfs = []
    for ano in range(2018, 2024):
        caminho = f'../data/MortalidadeBrasil/ETLSIM.DORES_PE_{ano}_t.csv'
        df = pd.read_csv(caminho, encoding='latin-1', sep=',', dtype={'CODMUNRES': str})
        dfs.append(df)
        print(f"✅ Dados de {ano} carregados | Registros: {len(df):,}")

    df_completo = pd.concat(dfs)
    print("\n📊 DATASET CONSOLIDADO")
    print(f"Registros totais: {len(df_completo):,}")
    print(f"Período: {df_completo['DTOBITO'].min()} a {df_completo['DTOBITO'].max()}")

    return df_completo

def preprocessar_dados_chuva():
    # 1. Carregar dados brutos
    df1 = pd.read_csv(
        '../data/Chuvas/Chuvas18-21.csv',
        sep=',',
        decimal=',',
        encoding='utf-8',
        dtype=str
    )

    df2 = pd.read_csv(
        '../data/Chuvas/Chuvas21-25.csv',
        sep=',',
        decimal=',',
        encoding='utf-8',
        dtype=str
    )

    # 2. Unificação e limpeza
    df_chuva = pd.concat([df1, df2], ignore_index=True)

    # 3. Padronização temporal
    meses = {
        'jan./': '01/', 'fev./': '02/', 'mar./': '03/', 'abr./': '04/',
        'mai./': '05/', 'jun./': '06/', 'jul./': '07/', 'ago./': '08/',
        'set./': '09/', 'out./': '10/', 'nov./': '11/', 'dez./': '12/'
    }

    for old, new in meses.items():
        df_chuva['Mês/Ano'] = df_chuva['Mês/Ano'].str.replace(old, new)

    # 4. Normalização de nomes geográficos
    mapeamento_cidades = {
        'Araçoiaba (Granja Cristo Redentor)': 'Araçoiaba',
        'Cabo (Barragem de Gurjaú)': 'Cabo de Santo Agostinho',
        'Cabo (Barragem de Suape)': 'Cabo de Santo Agostinho',
        'Cabo (Pirapama)': 'Cabo de Santo Agostinho',
        'Ipojuca (Suape) - PCD': 'Ipojuca',
        'Jaboatão (Cidade da Copa) - PCD': 'Jaboatão dos Guararapes',
        'Recife (Codecipe / Santo Amaro)': 'Recife',
        'Recife (Várzea)': 'Recife',
        'São Lourenço da Mata (Tapacurá)': 'São Lourenço da Mata'
    }

    df_chuva['Posto'] = df_chuva['Posto'].replace(mapeamento_cidades)

    # 5. Conversão numérica robusta
    day_cols = [str(i) for i in range(1,32)]
    for col in day_cols + ['Acumulado']:
        df_chuva[col] = (
            df_chuva[col]
            .str.replace('[^0-9,]', '', regex=True)
            .str.replace(',', '.')
            .replace('', '0')
            .astype(float)
        )

    # 6. Consistência final
    df_chuva = df_chuva.groupby(['Mês/Ano', 'Posto']).first().reset_index()

    return df_chuva

def processar_dados_chuva():
    """Processa e formata os dados pluviométricos"""
    print("\n🌧️ PROCESSANDO DADOS DE CHUVA")
    print("="*50)

    # Código de pré-processamento anterior (mantido)
    df_chuva = preprocessar_dados_chuva()
    print("\n✅ Cidades padronizadas:", df_chuva['Posto'].unique())
    print("✅ Formato temporal:", df_chuva['Mês/Ano'].unique()[:5])
    print("✅ Exemplo de dados:\n", df_chuva.sample(3))

    # Converter o formato wide para long (uma linha por dia/mês/cidade)
    df_chuva_reform = df_chuva.melt(
        id_vars=['Posto', 'Mês/Ano'],
        value_vars=[str(i) for i in range(1, 32)],
        var_name='Dia',
        value_name='Chuva_mm'
    )

    # Converter dia para inteiro
    df_chuva_reform['Dia'] = df_chuva_reform['Dia'].astype(int)

    return df_chuva_reform

def corrigir_df_rmr(df):
    # 1. Converter datas
    df['DTOBITO'] = pd.to_datetime(
        df['DTOBITO'].astype(str).str.zfill(8),
        format='%d%m%Y',
        errors='coerce'
    )

    # 2. Ajustar encoding
    df['ocor_MUNNOME'] = (
        df['ocor_MUNNOME']
        .str.encode('latin-1').str.decode('utf-8', errors='ignore')
        .replace({
            'SÃ£o LourenÃ§o da Mata': 'São Lourenço da Mata',
            'JaboatÃ£o dos Guararapes': 'Jaboatão dos Guararapes',
            'Ilha de ItamaracÃ¡': 'Ilha de Itamaracá',
            'AraÃ§oiaba': 'Araçoiaba'
        })
    )

    # 3. Mapear variáveis categóricas
    df['SEXO'] = df['SEXO'].map({
        1: 'Masculino',
        2: 'Feminino',
        0: 'Ignorado'
    })

    # 4. Corrigir idade (supondo que idade está em dias)
    if 'idade_obito_anos' not in df.columns:
        df['idade_obito_anos'] = df['IDADE'] // 365  # Se IDADE estiver em dias

    # 5. Remover colunas vazias
    cols_vazias = [col for col in df.columns if df[col].isna().all()]
    df = df.drop(columns=cols_vazias)

    return df

# =====================================
# DADOS DE REFERÊNCIA (RMR - Pernambuco)
# =====================================
# Códigos IBGE dos municípios da RMR (6 dígitos, como aparece no seu DataFrame)
codigos_rmr_6digitos = [
    '261160',  # Recife
    '260790',  # Jaboatão dos Guararapes
    '260960',  # Olinda
    '261070',  # Paulista
    '260290',  # Cabo de Santo Agostinho
    '260345',  # Camaragibe
    '261370',  # São Lourenço da Mata
    '260680',  # Igarassu
    '260005',  # Abreu e Lima
    '260720',  # Ipojuca
    '260940',  # Moreno
    '260105',  # Araçoiaba
    '260775',  # Itapissuma
    '260760'   # Itamaracá
]

# Nomes correspondentes aos códigos
nomes_municipios = {
    '260005': 'Abreu e Lima',
    '260105': 'Araçoiaba',
    '260290': 'Cabo de Santo Agostinho',
    '260345': 'Camaragibe',
    '260680': 'Igarassu',
    '260720': 'Ipojuca',
    '260760': 'Itamaracá',
    '260775': 'Itapissuma',
    '260790': 'Jaboatão dos Guararapes',
    '260940': 'Moreno',
    '260960': 'Olinda',
    '261070': 'Paulista',
    '261160': 'Recife',
    '261370': 'São Lourenço da Mata'
}

def processar_dados_mortalidade(df):
    # Deixar apenas Pernambuco
    df_pe = df[df['ocor_SIGLA_UF'] == 'PE']

    # Criar uma coluna temporária com os 6 primeiros dígitos de CODMUNOCOR
    df_pe['CODMUNOCOR'] = df_pe['CODMUNOCOR'].astype(str).str[:6]

    # Filtrar apenas os municípios da RMR
    df_rmr = df_pe[df_pe['CODMUNOCOR'].isin(codigos_rmr_6digitos)].copy()

    # Adicionar nomes dos municípios
    df_rmr['NOME_MUNICIPIO'] = df_rmr['CODMUNOCOR'].map(nomes_municipios)

    # Aplicar correções
    df_rmr_corrigido = corrigir_df_rmr(df_rmr)

    # Verificar resultados
    print("\n🔍 Datas convertidas (exemplo):", df_rmr_corrigido['DTOBITO'].head(3))
    print("\n🌆 Municípios normalizados:", df_rmr_corrigido['ocor_MUNNOME'].unique())
    print("\n👥 Distribuição por sexo:\n", df_rmr_corrigido['SEXO'].value_counts())

    # Adicionar ao código de correção (etapa 2)
    # Modificar df_rmr_corrigido em vez de df
    df_rmr_corrigido['ocor_MUNNOME'] = df_rmr_corrigido['ocor_MUNNOME'].replace('Ilha de Itamaracá', 'Itamaracá')

    df_analise = df_rmr_corrigido[df_rmr_corrigido['SEXO'].isin(['Masculino', 'Feminino'])]

    # Criar colunas de junção
    df_rmr_corrigido['MES_ANO'] = df_rmr_corrigido['DTOBITO'].dt.strftime('%m/%Y')
    df_rmr_corrigido['DIA'] = df_rmr_corrigido['DTOBITO'].dt.day

    # Criar colunas de junção no df_rmr_corrigido
    df_rmr_corrigido['MES_ANO'] = df_rmr_corrigido['DTOBITO'].dt.strftime('%m/%Y')
    df_rmr_corrigido['DIA'] = df_rmr_corrigido['DTOBITO'].dt.day

    return df_rmr_corrigido

# ========================================================
#               FUNÇÃO PRINCIPAL
# ========================================================

# Etapa 1: Carregar dados
df_mortalidade = carregar_dados_mortalidade()
df_chuva = processar_dados_chuva()

# Etapa 2: Processamento e merge
df_processado = processar_dados_mortalidade(df_mortalidade)
df_final = pd.merge(
    df_processado,
    df_chuva,
    left_on=['ocor_MUNNOME', 'MES_ANO', 'DIA'],
    right_on=['Posto', 'Mês/Ano', 'Dia'],
    how='left'
)


📂 CARREGANDO DADOS DE MORTALIDADE


/tmp/ipykernel_49795/2110664843.py:21: DtypeWarning: Columns (72) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho, encoding='latin-1', sep=',', dtype={'CODMUNRES': str})


✅ Dados de 2018 carregados | Registros: 62,011
✅ Dados de 2019 carregados | Registros: 64,295


/tmp/ipykernel_49795/2110664843.py:21: DtypeWarning: Columns (71,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho, encoding='latin-1', sep=',', dtype={'CODMUNRES': str})


✅ Dados de 2020 carregados | Registros: 76,574
✅ Dados de 2021 carregados | Registros: 80,717


...

✅ Dados de 2022 carregados | Registros: 72,011


/tmp/ipykernel_49795/2110664843.py:21: DtypeWarning: Columns (53,66,68) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho, encoding='latin-1', sep=',', dtype={'CODMUNRES': str})


✅ Dados de 2023 carregados | Registros: 68,527

📊 DATASET CONSOLIDADO
Registros totais: 424,135
Período: 1012018 a 31122023

🌧️ PROCESSANDO DADOS DE CHUVA

✅ Cidades padronizadas: ['Abreu e Lima' 'Araçoiaba' 'Cabo' 'Cabo de Santo Agostinho' 'Camaragibe'
 'Goiana (Itapirema - IPA)' 'Goiana - PCD' 'Igarassu'
 'Igarassu (Bar.Catucá)' 'Igarassu (Usina São José)' 'Ipojuca' 'Itamaracá'
 'Itapissuma' 'Jaboatão dos Guararapes'
 'Jaboatão dos Guararapes (Bar.Duas Unas)' 'Moreno' 'Olinda'
 'Olinda (Academia Santa Gertrudes)' 'Olinda (Alto da Bondade)' 'Paulista'
 'Recife' 'Recife (Alto da Brasileira)' 'São Lourenço da Mata'
 'Olinda (Jardim Atlântico)']
✅ Formato temporal: ['01/2018' '01/2019' '01/2020' '01/2021' '01/2022']
✅ Exemplo de dados:
       Mês/Ano                              Posto Unnamed: 0 Código     1    2  \
140   01/2024                       Abreu e Lima        928    198  68.0  1.0   
1574  09/2024                             Recife       1186    196   0.0  0.0   
1711  10/202

/tmp/ipykernel_49795/2110664843.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pe['CODMUNOCOR'] = df_pe['CODMUNOCOR'].astype(str).str[:6]



🔍 Datas convertidas (exemplo): 1   2018-04-16
4   2018-09-08
5   2018-01-10
Name: DTOBITO, dtype: datetime64[ns]

🌆 Municípios normalizados: ['São Lourenço da Mata' 'Recife' 'Jaboatão dos Guararapes'
 'Ilha de Itamaracá' 'Ipojuca' 'Cabo de Santo Agostinho' 'Olinda'
 'Camaragibe' 'Paulista' 'Araçoiaba' 'Itapissuma' 'Abreu e Lima'
 'Igarassu' 'Moreno']

👥 Distribuição por sexo:
 SEXO
Masculino    120502
Feminino     108137
Ignorado        134
Name: count, dtype: int64


### 📊 Comparação Diária: Óbitos e Chuvas na RMR

In [9]:
def comparar_rmr_dia(data_input, cidade, df, top_n=10):
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns

    rmr_cidades = [
        'Recife', 'Igarassu', 'Camaragibe', 'Araçoiaba', 'Cabo de Santo Agostinho',
        'São Lourenço da Mata', 'Itamaracá', 'Jaboatão dos Guararapes', 'Paulista',
        'Ipojuca', 'Moreno', 'Olinda', 'Abreu e Lima', 'Itapissuma'
    ]

    try:
        data = pd.to_datetime(data_input).date()
    except:
        print("❌ Data inválida. Use o formato 'YYYY-MM-DD'.")
        return

    df_rmr_dia = df[
        (df['ocor_MUNNOME'].isin(rmr_cidades)) &
        (pd.to_datetime(df['data_obito']).dt.date == data)
    ]

    if df_rmr_dia.empty:
        print(f"⚠️ Nenhum dado encontrado para a RMR em {data}.")
        return

    dados_cidade = df_rmr_dia[df_rmr_dia['ocor_MUNNOME'].str.upper() == cidade.upper()]

    if dados_cidade.empty:
        print(f"⚠️ Nenhum dado encontrado para {cidade.title()} em {data}.")
        return

    # Preparar dados
    causas = dados_cidade['CAUSABAS'].value_counts().head(top_n)
    agrupado = df_rmr_dia.groupby('ocor_MUNNOME').agg({
        'DTOBITO': 'count',
        'Chuva_mm': 'mean'
    }).rename(columns={'DTOBITO': 'Total_Obitos', 'Chuva_mm': 'Media_Chuva_mm'}).fillna(0)

    rmr_existentes = [c for c in rmr_cidades if c in agrupado.index]
    agrupado = agrupado.loc[rmr_existentes]
    cores = ['red' if cidade.lower() != mun.lower() else 'darkblue' for mun in agrupado.index]

    # Criar dois gráficos lado a lado
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

    # Pizza
    ax1.pie(causas, labels=causas.index, autopct='%1.1f%%', startangle=140,
            colors=sns.color_palette('Reds', n_colors=top_n))
    ax1.set_title(f'{cidade.title()} - {data}\nTop {top_n} Causas de Óbito (CAUSABAS)')

    # Barras + linha
    sns.barplot(x=agrupado.index, y=agrupado['Total_Obitos'], palette=cores, ax=ax2)
    ax2.set_ylabel('Total de Óbitos', color='black')
    ax2.set_xlabel('')
    ax2.set_xticklabels(agrupado.index, rotation=45, ha='right')

    ax3 = ax2.twinx()
    sns.lineplot(x=agrupado.index, y=agrupado['Media_Chuva_mm'], color='blue', marker='o', label='Chuva Média (mm)', ax=ax3)
    ax3.set_ylabel('Média de Chuva (mm)', color='blue')
    ax3.tick_params(axis='y', labelcolor='blue')

    ax2.set_title(f'Comparativo por Cidade na RMR - {data}')

    plt.tight_layout()
    plt.show()

    # Resumo
    print(f"\n📅 Data: {data}")
    print(f"🏙️ Cidade: {cidade.title()}")
    print(f"💀 Óbitos: {len(dados_cidade)}")
    print(f"☔ Chuva: {dados_cidade['Chuva_mm'].mean():.2f} mm")


# Interações e Gráficos

### 🔗 Correlação entre Chuva e Óbitos por Causas Infecciosas (CID-W)

In [10]:
df_final = df_final[df_final['CAUSABAS'].str.startswith(('W'), na=False)]
df_final = df_final.rename(columns={'CONTADOR': 'Total_Obitos'})

for cidade in df_final['ocor_MUNNOME'].unique():
    df_cidade = df_final[df_final['ocor_MUNNOME'] == cidade]
    corr_local = df_cidade[['Chuva_mm', 'Total_Obitos']].corr().iloc[0, 1]
    print(f"{cidade}: {corr_local:.2f}")

Recife: -0.14
Igarassu: -0.22
Camaragibe: -0.34
Araçoiaba: -0.17
Cabo de Santo Agostinho: -0.19
São Lourenço da Mata: -0.27
Itamaracá: 0.18
Jaboatão dos Guararapes: -0.13
Paulista: -0.19
Ipojuca: -0.27
Moreno: -0.13
Olinda: -0.07
Abreu e Lima: 0.02
Itapissuma: -0.37


In [12]:
comparar_rmr_dia("2024-05-1", "Recife", df_final)

⚠️ Nenhum dado encontrado para a RMR em 2024-05-01.
